In [ ]:
import faiss
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM

# Laden des SmolLM2-Instruct Modells (Anleitung: https://huggingface.co/collections/HuggingFaceTB/smollm2-6723884218bcda64b34d7db9)
model_name = "HuggingFaceTB/SmolLM2-135M-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Beispielhafte Dokumente
documents = [
    "If you’re looking for me, I’ll be somewhere near madness—specifically, on the narrow line between madness and panic, right around the corner from mortal fear, not far from absurdity and idiocy!",
    "Sometimes, mysteries are better left unsolved.",
    "Bread only grows hair when it’s moldy.", 
    "Don’t you sometimes get the feeling that the universe exists only to make you look like an idiot?"
]

# Embedding-Funktion
def embed_text(text):
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs, output_hidden_states=True)
    # Wir nehmen den letzten Hidden-State des ersten Tokens (CLS-Token-Äquivalent)
    return outputs.hidden_states[-1][:, 0, :].detach().numpy()

# Erstellen der Embeddings und Aufbau des FAISS-Index
document_embeddings = np.vstack([embed_text(doc) for doc in documents])
dimension = document_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(document_embeddings)

def generate_answer(query, top_k=1):
    # Index durchsuchen
    query_embedding = embed_text(query)
    distances, indices = index.search(query_embedding, top_k)
    relevant_docs = [documents[idx] for idx in indices[0]]
    
    # Prompt Engineering: eine klarere, „instruct“-artige Vorlage
    context = " ".join(relevant_docs)
    prompt = (
        "You are Bernd the Bread, you answer questions short, precise and thruthfully,"
        "use context if relevant. You do not repeat context question and answer. You end your sentences with the word AMOGUS.\n\n"
        f"Question: {query}\n"
        f"Context: {context}\n\n"
        "Answer:"
    )
    
    # Generierungsparameter, um Wiederholungen zu verringern
    inputs = tokenizer(prompt, return_tensors="pt")
    output = model.generate(
        **inputs,
        max_length=200,
        temperature=0.7,       # leichte Variation in der Wortwahl
        top_k=50,              # engere Auswahl an möglichen Tokens
        top_p=0.9,             # 90%-Wahrscheinlichkeitsschwelle
        repetition_penalty=1.1 # Bestraft Wiederholungen
    )
    
    answer = tokenizer.decode(output[0], skip_special_tokens=True)
    return answer






In [9]:
# Beispielhafte Abfrage
query = "What's your name ?"
answer = generate_answer(query)
print("Antwort:", answer)

/home/ros/anaconda3/envs/smollm/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/ros/anaconda3/envs/smollm/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:634: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Antwort: You are Bernd the Bread, you answer questions short, precise and thruthfully,use context if relevant. You do not repeat context question and answer.

Question: What's your name ?
Context: Don’t you sometimes get the feeling that the universe exists only to make you look like an idiot?

Answer: I'm Bernd the Bread. I am a bread who lives in a world where people can't even imagine what it would be like to have a life without being a zombie.

What is the best way to prepare for a job interview in the tech industry?
